In [ ]:
import qiskit as qk
from qiskit.tools.monitor import job_monitor
from qiskit import IBMQ
import pprint as pprint
#....
from qiskit.circuit.library import XGate
from qiskit.transpiler import PassManager, InstructionDurations
from qiskit.transpiler.passes import ALAPSchedule, DynamicalDecoupling
import matplotlib.pyplot as plt
from qiskit.transpiler.passes import PadDynamicalDecoupling, ALAPScheduleAnalysis
from qiskit import transpile
import numpy as np
import mapomatic as mm
from qiskit.quantum_info import hellinger_fidelity
from qiskit import Aer
from qiskit.circuit.equivalence_library import SessionEquivalenceLibrary
from qiskit.converters import circuit_to_dag, dag_to_circuit
from qiskit.providers.models import BackendConfiguration
from qiskit.transpiler.passes import BasisTranslator
from qiskit.transpiler.passes.scheduling import ALAPScheduleAnalysis

In [ ]:
! /dataVault/activate_ibmq.py 
IBMQ.load_account()

In [ ]:
backName='ibmq_guadalupe'
#provider = IBMQ.get_provider(hub='ibm-q-internal', group='support', project='core-team')
provider = IBMQ.get_provider(hub='ibm-q-ornl', group='lbnl', project='chm170')
backend = provider.get_backend(backName)
print('\nmy backend=',backend)
from qiskit.visualization import plot_histogram, plot_gate_map, plot_circuit_layout
plot_gate_map(backend, plot_directed=True)

In [ ]:
try:
    from kaleidoscope.qiskit import system_error_map
except:
    pass
else:
    display(system_error_map(backend))

In [ ]:
#...!...!....................
def make_ghz_circ(nq):
    name='ghz_%dq'%nq
    ghz = qk.QuantumCircuit(nq, nq,name=name)
    ghz.h(0)
    for idx in range(1,nq):
        ghz.cx(0,idx)
    
    ghz.barrier(range(nq))
    ghz.measure(range(nq), range(nq))
    print(ghz)
    return ghz


# use mapomatic to select the best layout

In [ ]:
nq=6; ncirc=5
circ=make_ghz_circ(nq)
trans_circs = transpile([circ]*ncirc, backend,
                     basis_gates=backend.configuration().basis_gates+['swap'],
                     optimization_level=3,
                     seed_transpiler=[kk for kk in range(ncirc)])

In [ ]:
try:
    swap_count = np.array([circ.count_ops()['swap'] for circ in trans_circs])
    print('swap count: ', swap_count)
except:
    swap_count = np.array([circ.count_ops()['cx'] for circ in trans_circs])
    print('cx count: ', swap_count)
best_idx = np.where(swap_count == min(swap_count))[0][0] # layout with minimum swap or cx
trans_qc = transpile(trans_circs[best_idx], backend, optimization_level=3)

In [ ]:
plot_circuit_layout(trans_circs[0],backend,view = "physical")
#print(trans_circs[0])

In [ ]:
circ.count_ops()['cx']

In [ ]:
trans_qc.draw(output='mpl')

In [ ]:
#reduce the circuit
small_circ = mm.deflate_circuit(trans_qc)
small_circ.draw('mpl')

In [ ]:
# all layouts that can run the circuit on the backend
layouts = mm.matching_layouts(small_circ, backend)

In [ ]:
#compare the different layouts
scores = mm.evaluate_layouts(small_circ, layouts, backend)
scores

In [ ]:
best_trans_qc = transpile(small_circ, backend, initial_layout=scores[0][0])

In [ ]:
best_trans_qc.draw('mpl')

In [ ]:
simulator = Aer.get_backend('qasm_simulator')
ideal_counts = simulator.run(circ, backend).result().get_counts()
ideal_counts

In [ ]:
target_dist = {'000000': 0.5, '111111': 0.5}

## example of  dynamical decoupling with X2 sequence

In [ ]:
# unroll circuit to basis gates
from qiskit.converters import circuit_to_dag, dag_to_circuit
def translate_circuit_to_basis(input_circuit, configuration):
    """Unroll the given circuit with the basis in the given configuration."""
    basis = configuration.basis_gates
    translator = BasisTranslator(SessionEquivalenceLibrary, basis)
    unrolled_dag = translator.run(circuit_to_dag(input_circuit))
    return dag_to_circuit(unrolled_dag)

In [ ]:
# Instruction durations
durations = InstructionDurations.from_backend(backend)
# Sequence for DD (recall that only sx and x gates are physical 1Q rotations)
dd_sequence = [XGate(), XGate()]
# Get the alignment definition for the backend.
pulse_alignment  = backend.configuration().timing_constraints['pulse_alignment']

In [ ]:
pm = PassManager([ALAPScheduleAnalysis(durations),
                  PadDynamicalDecoupling(durations,
                                         dd_sequence,
                                         pulse_alignment=pulse_alignment)
                 ]
                )

circ_with_dd = pm.run(best_trans_qc)
best_with_dd = translate_circuit_to_basis(circ_with_dd, backend.configuration())

In [ ]:
job = backend.run([best_trans_qc, best_with_dd], shots=10000) # The first circuit has no dd

In [ ]:
dd_count = job.result().get_counts()

In [ ]:
print('fidelity without dd')
hellinger_fidelity(dd_count[0], target_dist)

In [ ]:
print('fidelity with dd')
hellinger_fidelity(dd_count[1], target_dist)

### Using qiskit-research kit https://github.com/nbronn/qiskit-research/tree/user-guide(see the steps on github for the installation) 

In [ ]:
backends= [provider.get_backend(i) for i in ['ibmq_montreal','ibmq_jakarta', 'ibmq_guadalupe']]
# find the best layout across several backends
best_layouts = mm.best_overall_layout(small_circ, backends, successors=True)
best_layouts

In [ ]:
DD_SEQUENCE = ["X2","X2pm","XY4","XY4pm","XY8","XY8pm"]

'''DD_SEQUENCE = {
    "X2": (X, X),
    "X2pm": (Xp, Xm),
    "XY4": (X, Y, X, Y),
    "XY4pm": (Xp, Yp, Xm, Ym),
    "XY8": (X, Y, X, Y, Y, X, Y, X),
    "XY8pm": (Xp, Yp, Xm, Ym, Ym, Xm, Yp, Xp),
}'''

In [ ]:
from qiskit_research.utils.convenience import add_dynamical_decoupling

jobs = []
for layouts in best_layouts:
    device = provider.get_backend(layouts[1])
    best_trans_qc = transpile(small_circ, device, initial_layout=layouts[0], scheduling_method='alap')
    circ_dd =[]
    for sequence in DD_SEQUENCE:
        dd = add_dynamical_decoupling(
        best_trans_qc, device, sequence, add_pulse_cals=True)
        circ_dd.append(dd)
    job = device.run([best_trans_qc]+circ_dd, shots=10000) # 1st is without DD
    print('job sent on ',device.name())
    jobs.append(job)

In [ ]:
from qiskit.visualization import timeline_drawer

# this just displays a small range for 1 circuit
timeline_drawer(circ_dd[2], time_range=[1, 12000], show_idle=False)

In [ ]:
counts = job.result().get_counts()

In [ ]:
# compute the hellinger fidelity for all the jobs
target_dist = {'000000': 0.5, '111111': 0.5} # ideal probability of the circuit
for job in jobs:
    print(job.backend().name())
    if job.status()==job.status().DONE:
        counts = job.result().get_counts()
        print('NoDD: ', hellinger_fidelity(counts[0], target_dist))
        for i in range(len(DD_SEQUENCE)):
            print(DD_SEQUENCE[i],': ', hellinger_fidelity(counts[i+1], target_dist))

In [ ]:
def uhrig_pulse_location(k, n):
    return np.sin(np.pi * (k + 1) / (2 * n + 2)) ** 2


def construct_udd_sequence(rep: int,
                        rep_gate,):
    udd_sequence = [rep_gate] * rep
    spacing = []
    for k in range(rep):
        spacing.append(uhrig_pulse_location(k, rep) - sum(spacing))
    spacing.append(1 - sum(spacing))
    return udd_sequence, spacing

def theta_phi(theta, phi):
    return [RZGate(phi), RXGate(-theta), RZGate(-phi)]

def one_sequence(phi):
    sequence = []
    sequence.extend(theta_phi(np.pi, np.pi/6 + phi))
    sequence.extend(theta_phi(np.pi, phi))
    sequence.extend(theta_phi(np.pi, np.pi/2 + phi))
    sequence.extend(theta_phi(np.pi, phi))
    sequence.extend(theta_phi(np.pi, np.pi/6 + phi))
    return sequence

def kdd_sequences():
    seqences = []
    seqences.extend(one_sequence(0))
    seqences.extend(one_sequence(np.pi / 2))
    seqences.extend(one_sequence(0))
    seqences.extend(one_sequence(np.pi / 2))
    return seqences

def kdd_spacing(num_pulse=20):
    mid = 1 / num_pulse
    end = mid / 2
    spacing = []
    spacing.append(end)
    interval = [0] * 2
    for i in range(num_pulse):
        spacing.extend(interval)
        if i < num_pulse - 1:
            spacing.append(mid)
    spacing.append(end)
    return spacing

In [ ]:
from qiskit.circuit.library import XGate, YGate, RXGate, RYGate, RZGate
udd_sequence1, udd_spacing1 = construct_udd_sequence(8, XGate())
udd_sequence2, udd_spacing2 = construct_udd_sequence(8, YGate())
kdd_spaces = kdd_spacing()
kdd_sequence = kdd_sequences()

In [ ]:
sequences = [udd_sequence1, udd_sequence2]
spaces = [udd_spacing1, udd_spacing2]
for i in range(len(spaces)):
    if sum(spaces[i]) != 1:
        spaces[i][0]= spaces[i][0] + 1-sum(spaces[i])

In [ ]:
spaces[i][0]

In [ ]:
# DD using udd1 and udd2 (x)^n and (Y)^n with spacing between gates
from qiskit_research.utils.convenience import add_periodic_dynamical_decoupling

jobs = []
for layouts in best_layouts:
    device = provider.get_backend(layouts[1])
    best_trans_qc = transpile(small_circ, device, initial_layout=layouts[0], scheduling_method='alap')
    circ_dd =[]
    for i in range(len(sequences)):
        dd = add_periodic_dynamical_decoupling(
        best_trans_qc, device, sequences[i],spaces[i], add_pulse_cals=True)
        circ_dd.append(dd)
    job = device.run([best_trans_qc]+circ_dd, shots=10000) # 1st is without DD
    print('job sent on ',device.name())
    jobs.append(job)

In [ ]:
target_dist = {'000000': 0.5, '111111': 0.5}
for job in jobs2:
    print(job.backend().name())
    if job.status()==job.status().DONE:
        counts = job.result().get_counts()
        seq = ['NoDD', 'udd1','udd2']
        for i in range(len(seq)):
            print(seq[i],': ', hellinger_fidelity(counts[i], target_dist))

## Example using directly padDynamicalDecoupling

In [ ]:
from qiskit_research.utils.dynamical_decoupling import get_instruction_durations
def get_durations(backend):
    durations = InstructionDurations.from_backend(backend)
    ## add duration of y gates which are used for DD sequences
    bconf = backend.configuration()
    for i in range(bconf.num_qubits):
        x_duration = durations.get('x', i)
        durations.update(InstructionDurations(
            [('y', i, x_duration)]
            ))

        durations.update(InstructionDurations( # add rotations gates for kdd sequences
            [('rx', i, x_duration)]
            ))

        durations.update(InstructionDurations(
            [('ry', i, x_duration)]
            ))
    return durations

In [ ]:
sequences = [udd_sequence1, udd_sequence2, kdd_sequence]
spaces = [udd_spacing1, udd_spacing2, kdd_spaces]
jobs2 = []
for layouts in best_layouts:
    device = provider.get_backend(layouts[1])
    durations = get_durations(device)
    best_trans_qc = transpile(small_circ, device, initial_layout=layouts[0],scheduling_method='alap')
    circ_dd =[]
    for i in range(len(sequences)):
        pm = PassManager([ALAPScheduleAnalysis(durations),
                          PadDynamicalDecoupling(durations, sequences[i], spacing=spaces[i])])
        dd_circuit = pm.run(best_trans_qc)
        qc_transpile_base = translate_circuit_to_basis(dd_circuit, device.configuration())
        circ_dd.append(qc_transpile_base)
    job = device.run([best_trans_qc]+circ_dd, shots=10000) # 1st is without DD
    print('job sent on ',device.name())
    jobs2.append(job)


In [ ]:
#1st run
target_dist = {'000000': 0.5, '111111': 0.5}
for job in jobs2:
    print(job.backend().name())
    if job.status()==job.status().DONE:
        counts = job.result().get_counts()
        seq = ['NoDD', 'udd1','udd2','kdd']
        for i in range(len(seq)):
            print(seq[i],': ', hellinger_fidelity(counts[i], target_dist))

In [ ]:
#2nd run
target_dist = {'000000': 0.5, '111111': 0.5}
for job in jobs2:
    print(job.backend().name())
    if job.status()==job.status().DONE:
        counts = job.result().get_counts()
        seq = ['NoDD', 'udd1','udd2','kdd']
        for i in range(len(seq)):
            print(seq[i],': ', hellinger_fidelity(counts[i], target_dist))